## Custom Model

### PyTorch
First, define your model.

In [1]:
import numpy as np
import torch
from torch import nn


class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3)
        self.relu1 = torch.nn.ReLU()
        linear_input_size = np.power((224 - 3 + 2 * 0) / 1 + 1, 2) * 2
        self.linear = torch.nn.Linear(int(linear_input_size), 1000)
        self.relu2 = torch.nn.ReLU()  # can't get named ReLU output otherwise

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        x = self.relu2(x)
        return x



You can define your own preprocessing to convert filepaths into (normalized) model inputs.
We'll use a standardly defined function which normalizes to the ImageNet mean with an image size of 224 x 224.

In [2]:
import functools
from model_tools.activations.pytorch import load_preprocess_images

preprocessing = functools.partial(load_preprocess_images, image_size=224)

Then, using the PytorchWrapper, convert your model into an activations model.
Activations models let us extract activations from any layer in the model in response to inputs.

In [3]:
from model_tools.activations.pytorch import PytorchWrapper

activations_model = PytorchWrapper(identifier='my-model', model=MyModel(), preprocessing=preprocessing)

Model candidates in Brain-Score have to follow the [model_interface](https://github.com/brain-score/brain-score/blob/master/brainscore/model_interface.py).
This for instance involves deciding what layers map onto cortical regions.
If you want to use the standard commitments, use the `ModelCommitment`.
To map layers onto regions, `ModelCommitment` determines the best layer empirically by scoring all layers on public benchmarks.

In [4]:
from brainscore import score_model
from model_tools.brain_transformation import ModelCommitment

model = ModelCommitment(identifier='my-model', activations_model=activations_model,
                        # specify layers to consider
                        layers=['conv1', 'relu1', 'relu2'])
# The score_model will score the model on the specified benchmark.
# When the model is asked to output activations for the IT region, it will first search for the best layer
# and then only output this layer's activations.
score = score_model(model_identifier=model.identifier, model=model,
                    benchmark_identifier='dicarlo.MajajHong2015public.IT-pls')
print(score)


Loading lookup from /braintree/home/msch/brainio_collection/brainio_collection/lookup.csv


cross-validation: 100%|██████████| 10/10 [25:19<00:00, 151.96s/it]


<xarray.Score (aggregation: 2)>
array([0.07126698, 0.00550245])
Coordinates:
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:                   <xarray.Score (aggregation: 2)>\narray([0.21778479...
    ceiling:               <xarray.Score (aggregation: 2)>\narray([0.81579938...
    model_identifier:      my-model
    benchmark_identifier:  dicarlo.MajajHong2015public.IT-pls


### Tensorflow (-Slim)
First, define your model (its endpoints) and its preprocessing function.

In [5]:
from model_tools.activations.tensorflow import load_resize_image
import tensorflow as tf
slim = tf.contrib.slim
tf.reset_default_graph()

image_size = 224
placeholder = tf.placeholder(dtype=tf.string, shape=[64])
preprocess = lambda image_path: load_resize_image(image_path, image_size)
preprocess = tf.map_fn(preprocess, placeholder, dtype=tf.float32)

with tf.variable_scope('my_model', values=[preprocess]) as sc:
    end_points_collection = sc.original_name_scope + '_end_points'
    # Collect outputs for conv2d, fully_connected and max_pool2d.
    with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                        outputs_collections=[end_points_collection]):
        net = slim.conv2d(preprocess, 64, [11, 11], 4, padding='VALID', scope='conv1')
        net = slim.max_pool2d(net, [5, 5], 5, scope='pool1')
        net = slim.max_pool2d(net, [3, 3], 2, scope='pool2')
        net = slim.flatten(net, scope='flatten')
        net = slim.fully_connected(net, 1000, scope='logits')
        endpoints = slim.utils.convert_collection_to_dict(end_points_collection)

session = tf.Session()
session.run(tf.initialize_all_variables())

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Then, using the TensorflowSlimWrapper, convert your model into an activations model.
Activations models let us extract activations from any layer in the model in response to inputs.

In [6]:
from model_tools.activations.tensorflow import TensorflowSlimWrapper

activations_model_tf = TensorflowSlimWrapper(identifier='tf-custom', labels_offset=0,
                                             endpoints=endpoints, inputs=placeholder, session=session)

Model candidates in Brain-Score have to follow the [model_interface](https://github.com/brain-score/brain-score/blob/master/brainscore/model_interface.py).
This for instance involves deciding what layers map onto cortical regions.
If you want to use the standard commitments, use the `ModelCommitment`.
To map layers onto regions, `ModelCommitment` determines the best layer empirically by scoring all layers on public benchmarks.

In [7]:
from brainscore import score_model
from model_tools.brain_transformation import ModelCommitment

model = ModelCommitment(identifier='tf-custom', activations_model=activations_model_tf,
                        # specify layers to consider
                        layers=['my_model/conv1', 'my_model/pool1', 'my_model/pool2'])

score = score_model(model_identifier=model.identifier, model=model,
                    benchmark_identifier='dicarlo.MajajHong2015public.IT-pls')
print(score)


layers:   0%|          | 0/3 [00:00<?, ?it/s]



layer principal components:   0%|          | 0/3 [00:00<?, ?it/s]

layer principal components:  33%|███▎      | 1/3 [00:00<00:00,  2.56it/s]

layer principal components:  67%|██████▋   | 2/3 [00:01<00:00,  1.63it/s]

layer principal components: 100%|██████████| 3/3 [00:15<00:00,  5.32s/it]



layers: 100%|██████████| 3/3 [02:14<00:00, 44.82s/it]


cross-validation: 100%|██████████| 10/10 [00:24<00:00,  2.47s/it]

<xarray.Score (aggregation: 2)>
array([0.28334572, 0.00415787])
Coordinates:
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:                   <xarray.Score (aggregation: 2)>\narray([0.43425187...
    ceiling:               <xarray.Score (aggregation: 2)>\narray([0.81579938...
    model_identifier:      tf-custom
    benchmark_identifier:  dicarlo.MajajHong2015public.IT-pls


All of these steps are sample implementations and as long as your model implements the model_interface,
it does not matter how you get there.

## Pre-defined models

Scoring a model on neural data can be done in a single line using the `score_model` method and the `brain_translated_pool`.
Pre-defined layers of a model will be used to retrieve the activations.
Just like with the model implementations, the result of this method call will be cached 
so that it only needs to be computed once.


In [8]:
from brainscore import score_model
from candidate_models.model_commitments import brain_translated_pool

identifier = 'alexnet'
model = brain_translated_pool[identifier]
score = score_model(model_identifier=identifier, model=model, benchmark_identifier='dicarlo.MajajHong2015public.IT-pls')
print(score)


alexnet is accessed again and reloaded


cross-validation: 100%|██████████| 10/10 [03:13<00:00, 19.35s/it]

<xarray.Score (aggregation: 2)>
array([0.50478986, 0.00269386])
Coordinates:
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:                   <xarray.Score (aggregation: 2)>\narray([0.57961375...
    ceiling:               <xarray.Score (aggregation: 2)>\narray([0.81579938...
    model_identifier:      alexnet
    benchmark_identifier:  dicarlo.MajajHong2015public.IT-pls


A score typically comes with an estimate of the center (e.g. mean) and error (e.g. standard error of the mean).
These values are aggregations over splits and often neuroids, and ceiled by the benchmark ceiling.

Check out https://github.com/brain-score/brain-score/blob/master/examples/benchmarks.ipynb for more details.

Also note that all these scores were computed on the publicly available data.
To test models on the full set of benchmarks (including held-out private data), 
please submit them on www.Brain-Score.org.